In [1]:
import os
import sys
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

import onnx
from onnxsim import simplify
import onnxruntime
from onnxruntime.quantization import (quantize_dynamic,
                                      QuantType,
                                      QuantFormat,
                                      quantize_static,
                                      CalibrationDataReader)

sys.path.append('../')
from platforms.core.config import cfg
from siamfcpp.pipeline.utils import (cxywh2xywh, get_crop,
                                     get_subwindow_tracking,
                                     xywh2cxywh, xyxy2cxywh)
from siamfcpp.model.common_opr.common_block import xcorr_depthwise

In [2]:
def to_bchw(im_patch):
    im_patch = im_patch.transpose(2, 0, 1)
    im_patch = im_patch[np.newaxis, :, :, :]
    return im_patch.astype(np.float32)

### Static

*backbone_init*

In [6]:
calibration_image_folder = 'C:\\Users\\isd.illia.maliha\\work\\sorted_datasets\\background'
main_dataset_folder = '../datasets'

network_name = 'backbone_init'

# model = os.path.join("../models/onnx", network_name+'.onnx')
dataset_folder = os.path.join(main_dataset_folder, network_name)

In [9]:
def preprocess_images(images_folder: str, size_limit=0):
    image_names = os.listdir(images_folder)
    
    if size_limit > 0 and len(image_names) >= size_limit:
        batch_filenames = [image_names[i] for i in range(size_limit)]
    else:
        batch_filenames = image_names
        
    unconcatenated_batch_data = []


    for idx, image_name in enumerate(tqdm_notebook(batch_filenames[:])):
        image_filepath = images_folder + "/" + image_name
        image = cv2.imread(image_filepath).astype(np.float32)
        
        h,w,_ = image.shape
        x = np.random.randint(0,0.7*w)
        y = np.random.randint(0,0.7*h)
        ww = np.random.randint(25,w-x)
        hh = np.random.randint(25,h-y)
        
        box = xywh2cxywh([x,y,ww,hh])
        target_pos, target_sz = box[:2], box[2:]

        avg_chans = np.mean(image, axis=(0, 1))

        im_z_crop, _ = get_crop(
            image,
            target_pos,
            target_sz,
            127,
            avg_chans=avg_chans,
            context_amount=0.5,
            func_get_subwindow=get_subwindow_tracking,
        )

        im_z_crop = to_bchw(im_z_crop)

        np.save(os.path.join(dataset_folder, network_name, str(idx).zfill(7)+'.npy'), im_z_crop)
        with open(os.path.join(dataset_folder, network_name+'.txt'), 'a') as fff:
            fff.write(network_name + '/' + str(idx).zfill(7)+'.npy' + '\n')

In [10]:
preprocess_images(calibration_image_folder)

C:\Users\isd.illia.maliha\AppData\Local\Temp\ipykernel_4672\2113149029.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, image_name in enumerate(tqdm_notebook(batch_filenames[:])):


  0%|          | 0/882 [00:00<?, ?it/s]

-------

---------------------------

-----------------------------

*backbone*

In [11]:
calibration_image_folder = 'C:\\Users\\isd.illia.maliha\\work\\sorted_datasets\\background'
main_dataset_folder = '../datasets'

network_name = 'backbone'

# model = os.path.join("../models/onnx", network_name+'.onnx')
dataset_folder = os.path.join(main_dataset_folder, network_name)

In [12]:
def preprocess_images(images_folder: str, size_limit=0):
    image_names = os.listdir(images_folder)
    
    if size_limit > 0 and len(image_names) >= size_limit:
        batch_filenames = [image_names[i] for i in range(size_limit)]
    else:
        batch_filenames = image_names
        
    unconcatenated_batch_data = []


    for idx, image_name in enumerate(tqdm_notebook(batch_filenames[:])):
        image_filepath = images_folder + "/" + image_name
        image = cv2.imread(image_filepath).astype(np.float32)
        
        h,w,_ = image.shape
        x = np.random.randint(0,0.7*w)
        y = np.random.randint(0,0.7*h)
        ww = np.random.randint(25,w-x)
        hh = np.random.randint(25,h-y)
        
        box = xywh2cxywh([x,y,ww,hh])
        target_pos, target_sz = box[:2], box[2:]

        avg_chans = np.mean(image, axis=(0, 1))

        im_x_crop, scale_x = get_crop(
            image,
            target_pos,
            target_sz,
            127,
            x_size=303,
            avg_chans=avg_chans,
            context_amount=cfg.context_amount,
            func_get_subwindow=get_subwindow_tracking,
        )

        im_x_crop = to_bchw(im_x_crop)

        np.save(os.path.join(dataset_folder, network_name, str(idx).zfill(7)+'.npy'), im_x_crop)
        with open(os.path.join(dataset_folder, network_name+'.txt'), 'a') as fff:
            fff.write(network_name + '/' + str(idx).zfill(7)+'.npy' + '\n')

In [13]:
preprocess_images(calibration_image_folder)

C:\Users\isd.illia.maliha\AppData\Local\Temp\ipykernel_4672\2079499065.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, image_name in enumerate(tqdm_notebook(batch_filenames[:])):


  0%|          | 0/882 [00:00<?, ?it/s]

-----------------------

--------------------

------------------

*head*

In [13]:
backbone_init_fp32 = "../models/onnx/backbone_init.onnx"
backbone_fp32 = "../models/onnx/backbone.onnx"

In [14]:
calibration_image_folder = 'C:\\Users\\isd.illia.maliha\\work\\sorted_datasets\\background'
main_dataset_folder = '../datasets'

network_name = 'head'

# model = os.path.join("../models/onnx", network_name+'.onnx')
dataset_folder = os.path.join(main_dataset_folder, network_name)

In [15]:
def preprocess_images(backbone_init_path: str, backbone_path: str, images_folder: str, size_limit=0):
    
    bone_init = onnxruntime.InferenceSession(backbone_init_path, providers=['CPUExecutionProvider'])
    bone = onnxruntime.InferenceSession(backbone_path, providers=['CPUExecutionProvider'])
    
    image_names = os.listdir(images_folder)
    
    if size_limit > 0 and len(image_names) >= size_limit:
        batch_filenames = [image_names[i] for i in range(size_limit)]
    else:
        batch_filenames = image_names
        
    in1,in2 = [],[]
    

    for idx, image_name in enumerate(tqdm_notebook(batch_filenames[:])):
        image_filepath = images_folder + "/" + image_name
        image = cv2.imread(image_filepath).astype(np.float32)
        
        h,w,_ = image.shape
        x = np.random.randint(0,0.7*w)
        y = np.random.randint(0,0.7*h)
        ww = np.random.randint(25,w-x)
        hh = np.random.randint(25,h-y)
        
        box = xywh2cxywh([x,y,ww,hh])
        target_pos, target_sz = box[:2], box[2:]

        avg_chans = np.mean(image, axis=(0, 1))

        im_z_crop, _ = get_crop(
            image,
            target_pos,
            target_sz,
            127,
            avg_chans=avg_chans,
            context_amount=0.5,
            func_get_subwindow=get_subwindow_tracking,
        )
        im_z_crop = to_bchw(im_z_crop)
        
        c_z_k, r_z_k = bone_init.run(None, {'input': im_z_crop})
    
    

        im_x_crop, scale_x = get_crop(
            image,
            target_pos,
            target_sz,
            127,
            x_size=303,
            avg_chans=avg_chans,
            context_amount=cfg.context_amount,
            func_get_subwindow=get_subwindow_tracking,
        )
        im_x_crop = to_bchw(im_x_crop)
    
        c_x, r_x = bone.run(None, {'input': im_x_crop})
        
        c_out = xcorr_depthwise(torch.Tensor(c_x), torch.Tensor(c_z_k))
        r_out = xcorr_depthwise(torch.Tensor(r_x), torch.Tensor(r_z_k))
        

        np.save(os.path.join(dataset_folder, network_name, str(idx).zfill(7)+'_1.npy'), c_out.numpy().astype(np.float32))
        np.save(os.path.join(dataset_folder, network_name, str(idx).zfill(7)+'_2.npy'), r_out.numpy().astype(np.float32))
        
        with open(os.path.join(dataset_folder, network_name+'.txt'), 'a') as fff:
            fff.write(network_name + '/' + str(idx).zfill(7)+'_1.npy' + ' ' + network_name + '/' + str(idx).zfill(7)+'_2.npy' + '\n')

In [16]:
preprocess_images(backbone_init_fp32, backbone_fp32, calibration_image_folder)

C:\Users\isd.illia.maliha\AppData\Local\Temp\ipykernel_15468\2017415152.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, image_name in enumerate(tqdm_notebook(batch_filenames[:])):


  0%|          | 0/882 [00:00<?, ?it/s]